In [10]:
import xnat
import pyxnat
from pyxnat import Interface
import xnat
import os
import pydicom
import tempfile
from tqdm import tqdm


In [ ]:
# #The URL of your XNAT
# xnat_host = os.environ['XNAT_HOST']

# #A temporary token which can be used as a username
# user = os.environ['XNAT_USER']

# #A temporary token which can be used as a password
# pwd = os.environ['XNAT_PASS']

In [ ]:
# session = xnat.connect(xnat_host, user=user, password=pwd)

In [11]:
session = xnat.connect('https://xnat.crmbm.univ-amu.fr', user='hdary')

In [12]:
my_project = session.projects["TERRA_BRAIN_EPINOV"]

check tag dicom for gender

check tag dicom for age

In [13]:
my_project.subjects

<XNATListing {(XNAT_CRMBM_S00106, 01_55_L_M): <SubjectData 01_55_L_M (XNAT_CRMBM_S00106)>, (XNAT_CRMBM_S00107, 01_45_B-N): <SubjectData 01_45_B-N (XNAT_CRMBM_S00107)>, (XNAT_CRMBM_S00108, 01_85_G-S): <SubjectData 01_85_G-S (XNAT_CRMBM_S00108)>, (XNAT_CRMBM_S00109, 01-59-P-V): <SubjectData 01-59-P-V (XNAT_CRMBM_S00109)>, (XNAT_CRMBM_S00114, 01-68-P-N): <SubjectData 01-68-P-N (XNAT_CRMBM_S00114)>, (XNAT_CRMBM_S00125, 01-75-R-S): <SubjectData 01-75-R-S (XNAT_CRMBM_S00125)>, (XNAT_CRMBM_S00127, 01-63-B-K): <SubjectData 01-63-B-K (XNAT_CRMBM_S00127)>, (XNAT_CRMBM_S00110, 01_77_B_B): <SubjectData 01_77_B_B (XNAT_CRMBM_S00110)>, (XNAT_CRMBM_S00111, 01_86_P-L): <SubjectData 01_86_P-L (XNAT_CRMBM_S00111)>, (XNAT_CRMBM_S00116, 01-83-P-G): <SubjectData 01-83-P-G (XNAT_CRMBM_S00116)>, (XNAT_CRMBM_S00122, 01_71_D-C): <SubjectData 01_71_D-C (XNAT_CRMBM_S00122)>, (XNAT_CRMBM_S00126, 01_57_FC): <SubjectData 01_57_FC (XNAT_CRMBM_S00126)>, (XNAT_CRMBM_S00113, 01_80_V_T): <SubjectData 01_80_V_T (XNAT_CRM

In [6]:
subject = my_project.subjects['01-38-F-G']
experiment = subject.experiments['202011161412']
scan = experiment.scans[0]

KeyError: 'Could not find ID/label 01-38-F-G in collection!'

In [ ]:
scan.dicom_dump()

In [7]:
file_obj = scan.resources['DICOM'].files[0]

In [8]:
with file_obj.open() as fin:
    data = pydicom.dcmread(fin, stop_before_pixels=True)
    patient_birth_date = data[0x010,0x030].value
    sex = data[0x010,0x040].value
    age = data[0x010,0x1010].value
    size = data[0x010,0x1020].value
    weight = data[0x010,0x1030].value
    unique_id = data[0x008,0x050].value


In [9]:
print(patient_birth_date)
print(sex)
print(age)
print(size)
print(weight)
print(unique_id)

19820913
M
038Y
1.82
86



In [64]:
subject.custom_variables['default']['patient_unique_id'] = unique_id

<CustomVariableGroup default {patient_unique_id (string): None}>

In [65]:
experiment.custom_variables['default']['patient_unique_id'] = unique_id

Exception in thread XNATpyKeepAliveThread:
Traceback (most recent call last):
  File "/home/hdary/miniconda3/lib/python3.8/site-packages/urllib3/connection.py", line 159, in _new_conn
    conn = connection.create_connection(
  File "/home/hdary/miniconda3/lib/python3.8/site-packages/urllib3/util/connection.py", line 61, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/home/hdary/miniconda3/lib/python3.8/socket.py", line 918, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -3] Temporary failure in name resolution

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/hdary/miniconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 670, in urlopen
    httplib_response = self._make_request(
  File "/home/hdary/miniconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 381, in _ma

In [94]:
subject.demographics.age = int(age.replace('Y',''))
# subject.demographics.height = size
# subject.demographics.weight = weight
subject.demographics.yob = patient_birth_date
subject.demographics.gender = sex.replace('M','male').replace('F','female')


script 

In [15]:
my_project = session.projects["TERRA_BRAIN_HBP-MOBILE"]

for s in tqdm(my_project.subjects.values()):
    print(s.id, s.label)

    if s.label == 'SF_T002':
        continue

    if s.demographics.yob != None:
        continue
    
    e = s.experiments[0]
    scan = e.scans[0]
    file_obj = scan.resources['DICOM'].files[0]

    with file_obj.open() as fin:
        data = pydicom.dcmread(fin, stop_before_pixels=True)
        patient_birth_date = data[0x010,0x030].value
        sex = data[0x010,0x040].value
        age = data[0x010,0x1010].value
        size = data[0x010,0x1020].value
        weight = data[0x010,0x1030].value

        try:
            s.demographics.age = int(age.replace('Y',''))
        except ValueError as e:
            print('Error setting age:', e)

        # try:
        #     # s.demographics.height = size
        #     pass
        # except Exception as e:
        #     print('Error setting height:', e)

        # try:
        #     # s.demographics.weight = weight
        #     pass
        # except Exception as e:
        #     print('Error setting weight:', e)

        try:
            s.demographics.yob = patient_birth_date
        except ValueError as e:
            print('Error setting year of birth:', e)

        try:
            s.demographics.gender = sex.replace('M','male').replace('F','female')
        except ValueError as e:
            print('Error setting gender:', e)

  0%|          | 0/3 [00:00<?, ?it/s]

XNAT_CRMBM_S00151 SF_T002
XNAT_CRMBM_S00074 AC_T022


 67%|██████▋   | 2/3 [00:07<00:03,  3.68s/it]

XNAT_CRMBM_S00149 ED_T025


100%|██████████| 3/3 [00:08<00:00,  2.70s/it]


Exception in thread XNATpyKeepAliveThread:
Traceback (most recent call last):
  File "/home/hdary/miniconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 426, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/home/hdary/miniconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 421, in _make_request
    httplib_response = conn.getresponse()
  File "/home/hdary/miniconda3/lib/python3.8/http/client.py", line 1348, in getresponse
    response.begin()
  File "/home/hdary/miniconda3/lib/python3.8/http/client.py", line 316, in begin
    version, status, reason = self._read_status()
  File "/home/hdary/miniconda3/lib/python3.8/http/client.py", line 277, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/home/hdary/miniconda3/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
  File "/home/hdary/miniconda3/lib/python3.8/ssl.py", line 1241, in recv_into
  